### 1 Data

In [15]:
import requests

# Define coordinates (latitude, longitude)
lat, lon = -6.9061920783542, 107.32333620661745

# Open-Elevation API endpoint
url =f"https://api.opentopodata.org/v1/aster30m?locations={lat},{lon}"

# Send GET request
response = requests.get(url)

# Parse JSON response
if response.status_code == 200:
    data = response.json()
    elevation = data["results"][0]["elevation"]
    print(f"Elevation: {elevation} meters")
else:
    print("Error:", response.status_code)

Elevation: 736.0 meters


### To txt

In [7]:
from osgeo import gdal
import numpy as np

# Open the GeoTIFF file
dataset = gdal.Open("output_AW3D30.tif")
band = dataset.GetRasterBand(1)  # Get the first band (elevation data)
elevation = band.ReadAsArray()  # Convert to NumPy array

# Get GeoTransform to map pixels to coordinates
gt = dataset.GetGeoTransform()
x_min, x_res, _, y_max, _, y_res = gt
x_max = x_min + (dataset.RasterXSize * x_res)
y_min = y_max + (dataset.RasterYSize * y_res)

# Create coordinate grid
lon = np.linspace(x_min, x_max, dataset.RasterXSize)
lat = np.linspace(y_max, y_min, dataset.RasterYSize)
lon_grid, lat_grid = np.meshgrid(lon, lat)

# Save as JSON
import json
data = []
for i in range(dataset.RasterYSize):
    for j in range(dataset.RasterXSize):
        data.append({
            "latitude": float(lat_grid[i, j]),
            "longitude": float(lon_grid[i, j]),
            "elevation": float(elevation[i, j])
        })

with open("output_AW3D30.json", "w") as f:
    json.dump(data, f, indent=4)

print("GeoTIFF extracted to output_AW3D30.json")

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/osgeo/gdal.py:314: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


GeoTIFF extracted to output_AW3D30.json


### Cek data

In [9]:
import json

# Load JSON file
with open("output_AW3D30.json", "r") as file:
    data = json.load(file)

# Check the number of entries
if isinstance(data, list):
    print(f"Total entries: {len(data)}")
elif isinstance(data, dict):
    print(f"Total keys: {len(data.keys())}")
else:
    print("Unknown JSON format")

Total entries: 6091936


In [13]:
import math

def lon_distance_meters(lon1, lon2, lat):
    """Calculate distance in meters between two longitudes at a given latitude."""
    meters_per_degree = 111320 * math.cos(math.radians(lat))
    return abs(lon2 - lon1) * meters_per_degree

# Given longitude values
lon1 = 107.31472222226273
lon2 = 107.31500009272578
latitude = -6.9  # Example: Bandung, Indonesia

# Calculate distance
distance_m = lon_distance_meters(lon1, lon2, latitude)
print(f"Distance: {distance_m:.2f} meters")

print(60000**2/distance_m**2)


Distance: 30.71 meters
3817553.4002595027
